In [38]:
# Importamos librerías 
import numpy as np
from functools import reduce
import time
import matplotlib.pyplot as plt
import scipy.stats as st     # Librería estadística
import pandas as pd
from scipy import optimize

In [39]:
suicidios = pd.read_excel('Suicidios_Estados.xlsx', sheet_name='Tabulado', header=None).dropna()
suicidios = suicidios.drop(columns=[2,3,4,6,7,8,10,11,12])
suicidios.columns=['Estados','2017','2018','2019']
suicidios.head()

,Estados,2017,2018,2019
5,Total,6559,6808,7223
6,Aguascalientes,140,140,157
7,Baja California,166,194,169
8,Baja California Sur,47,57,59
9,Campeche,74,69,95


In [54]:
mort = pd.read_excel('Mortalidad.xlsx', sheet_name='Tabulado', header=None).dropna()
mort = mort.drop(columns=[2,3,4,6,7,8,10,11,12])
mort.columns=['Estados','2017','2018','2019']
mort.head()

,Estados,2017,2018,2019
5,Total,703047,722611,747784
6,Aguascalientes,6158,6226,6768
7,Baja California,19073,20501,20924
8,Baja California Sur,3826,3447,3556
9,Campeche,4631,4835,5121


In [52]:
s_causa = pd.read_excel('Suicidios_Causas2.xlsx', sheet_name='Tabulado', header=None).dropna()
s_causa = s_causa.drop(columns=[2,3,5,6,8,9,11,12,14,15,17,18,20,21,23,24,26,27])
s_causa.columns=['Causas','2010','2011','2012','2013','2014','2015','2016','2017','2018']
# s_causa.head()
s_causa



,Causas,2010,2011,2012,2013,2014,2015,2016,2017,2018
5,Total,5012,5718,5549,5909,6337,6425,6370,6559,6808
6,"Ahorcamiento, estrangulamiento o sofocación",3847,4406,4291,4554,5000,5092,5145,5266,5486
7,Disparo de arma de fuego,547,595,524,574,529,573,491,518,515
8,Envenenamiento,460,523,508,556,577,512,517,517,506
9,Otras causas,158,194,226,225,231,248,217,258,301


In [63]:
s_edad = pd.read_excel('Suicidios_Edad.xlsx', sheet_name='Tabulado', header=None).dropna()
s_edad = s_edad.drop(columns=[2,3,5,6,8,9,11,12,14,15,17,18,20,21,23,24,26,27])
s_edad.columns=['Causas','2010','2011','2012','2013','2014','2015','2016','2017','2018']
# s_causa.head()
s_edad

,Causas,2010,2011,2012,2013,2014,2015,2016,2017,2018
5,Total,5012,5718,5549,5909,6337,6425,6370,6559,6808
6,10 a 14 años,187,249,208,174,243,236,228,209,229
7,15 a 19 años,685,859,826,780,761,861,774,795,800
8,20 a 24 años,761,916,877,875,1015,1010,1035,1027,1035
9,25 a 29 años,608,716,662,748,758,788,816,868,890
10,30 a 34 años,528,559,592,651,684,708,675,746,810
11,35 a 39 años,549,521,535,574,649,653,628,652,690
12,40 a 44 años,352,448,438,514,520,528,552,551,559
13,45 a 49 años,320,346,332,405,419,421,414,428,472
14,50 a 54 años,244,271,253,285,335,303,320,352,376
